# IMPORTS # 

In [ ]:
import json
import gc
import torchvision.transforms.functional as F
import torchvision.transforms as T
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import json
import os
import cv2
import numpy as np
from tqdm import tqdm 
from pathlib import Path
import torch
import cv2
import numpy as np
import json
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
from torchvision.models.detection.rpn import AnchorGenerator,RPNHead
from tqdm.notebook import tqdm
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Studio di xView 

In [ ]:
geojson_path = '/kaggle/input/xview-dataset/train_labels/xView_train.geojson' 
with open(geojson_path) as f:
    data = json.load(f)

class_ids = [feature['properties']['type_id'] for feature in data['features']]

df_classes = pd.DataFrame(class_ids, columns=['type_id'])

counts = df_classes['type_id'].value_counts().sort_index()

num_classes = df_classes['type_id'].nunique()

print(f"Totale classi: {num_classes}")
print(f"Totale oggetti annotati: {len(df_classes)}")
print("\n--- Conteggio per Class ID ---")
print(df_classes['type_id'].value_counts().sort_index(ascending=False).head(60)) #altrimenti me ne lista solamente 10 (5 top e 5 bottom)

# Tiling di xView # 

# Funzione per prelevare immagini xview #

trasformo il geojson in un json normale (per ogni img)

In [ ]:
def load_xview_data(geojson_path): # prendo geojson e creo un json/dizionario organizzato per id dell'img

    print("Caricamento GeoJSON... (può richiedere un po')")
    with open(geojson_path) as f:
        data = json.load(f)
    
    coords_by_img = {}
    
    for feature in tqdm(data['features']):
        props = feature['properties']
        img_name = props['image_id']
        bounds = [int(float(x)) for x in props['bounds_imcoords'].split(',')]
        type_id = props['type_id']
        
        if img_name not in coords_by_img:
            coords_by_img[img_name] = [] # dizionario di liste (key: img_name -> value: lista di annotazioni)
            
        coords_by_img[img_name].append({
            'bounds': bounds,
            'type_id': type_id
        })
        
    return coords_by_img

# Funzione di Tiling #
Usata per entrambi i dataset (train,val)

In [ ]:
def tile_dataset(image_list, output_dir, output_json_path, TILE_SIZE, STRIDE, coords_by_img):
    subset_data = []
    
    MIN_AREA = 16 * 16  
    MAX_TRUNCATION = 0.50 
    MIN_DIM = 4 
    # -------------------------------

    for img_name in tqdm(image_list):
        img_path = os.path.join(INPUT_IMAGES_DIR, img_name)
        img = cv2.imread(img_path)
        
        if img is None: 
            continue
                
        h, w, _ = img.shape
        original_boxes = coords_by_img[img_name]

        # Tiling Loop
        for y in range(0, h, STRIDE):
            for x in range(0, w, STRIDE):
                x2 = min(x + TILE_SIZE, w)
                y2 = min(y + TILE_SIZE, h)
                x1 = x2 - TILE_SIZE
                y1 = y2 - TILE_SIZE
                if x1 < 0: x1 = 0 
                if y1 < 0: y1 = 0
                
                # Estrai la tile
                tile_img = img[y1:y2, x1:x2]
                
                valid_objects = []
                for obj in original_boxes:
                    bx1, by1, bx2, by2 = obj['bounds']
                    
                    # Calcolo Area Originale (per il troncamento)
                    original_w = bx2 - bx1
                    original_h = by2 - by1
                    original_area = original_w * original_h
                    
                    # Intersezione (Box nel Tile)
                    ix1 = max(x1, bx1)
                    iy1 = max(y1, by1)
                    ix2 = min(x2, bx2) 
                    iy2 = min(y2, by2)
                    
                    if ix1 < ix2 and iy1 < iy2:
                        # Coordinate locali nella tile
                        nx1 = ix1 - x1
                        ny1 = iy1 - y1
                        nx2 = ix2 - x1
                        ny2 = iy2 - y1
                        
                        new_w = nx2 - nx1
                        new_h = ny2 - ny1
                        new_area = new_w * new_h
                        
                        
                        # Filtro Dimensione Minima (Pixel)
                        if new_w < MIN_DIM or new_h < MIN_DIM:
                            continue # Troppo sottile/piccolo
                            
                        # Filtro Area Minima
                        if new_area < MIN_AREA:
                            continue # Oggetto troppo piccolo per essere riconosciuto
                            
                        # 3. Filtro Troncamento 
                        visible_ratio = new_area / original_area
                        truncation = 1.0 - visible_ratio
                        
                        if truncation > MAX_TRUNCATION:
                            continue # Oggetto troppo tagliato
                        
                        # Se passa i filtri
                        valid_objects.append({
                            'bounds': [nx1, ny1, nx2, ny2],
                            'type_id': obj['type_id']
                        })
                
                # Salva solo se la tile ha oggetti validi sopravvissuti ai filtri
                if len(valid_objects) > 0:
                    tile_filename = f"{Path(img_name).stem}_tile_{x}_{y}.jpg"
                    save_path = os.path.join(output_dir, tile_filename)
                    cv2.imwrite(save_path, tile_img)
                    subset_data.append({
                        'img_name': tile_filename,
                        'objects': valid_objects
                    })

    print(f"Creati {len(subset_data)} tiles validi.")
                    
    with open(output_json_path, 'w') as f:
        json.dump(subset_data, f)

In [ ]:
# --- CONFIGURAZIONE ---
INPUT_IMAGES_DIR = '/kaggle/input/xview-dataset/train_images/train_images' 
INPUT_GEOJSON = '/kaggle/input/xview-dataset/train_labels/xView_train.geojson'

# Cartelle di output
TRAIN_DIR = '/kaggle/working/train_tiles'
VAL_DIR = '/kaggle/working/val_tiles'
TRAIN_JSON = '/kaggle/working/train_annotations.json'
VAL_JSON = '/kaggle/working/val_annotations.json'
TEST_JSON = '/kaggle/working/test_annotations.json'
TEST_GT_JSON = '/kaggle/working/test_gt_annotations.json'

# Pulizia cartelle
if os.path.exists(TRAIN_DIR): shutil.rmtree(TRAIN_DIR)
if os.path.exists(VAL_DIR): shutil.rmtree(VAL_DIR)
os.makedirs(TRAIN_DIR)
os.makedirs(VAL_DIR)

TILE_SIZE = 512
STRIDE = 400

def create_datasets(num_train, num_val, num_test):
    
    # 1. Carico TUTTE le immagini disponibili
    print("Caricamento lista immagini...")
    coords_by_img = load_xview_data(INPUT_GEOJSON) 
    
    available_images = list(coords_by_img.keys()) 
    total_available = len(available_images)
    
    print(f"Totale immagini nell'urna: {total_available}")
    
    # Controllo sicurezza
    if (num_train + num_val + num_test) > total_available:
        print("ERRORE: Hai chiesto più immagini di quelle disponibili!")
        return

    # Liste vuote da riempire
    train_imgs = []
    val_imgs = []
    test_imgs = []

    # ESTRAZIONE TRAINING SET 
    print(f"\n--- Estrazione casuale di {num_train} immagini per il TRAIN ---")
    random.seed(42) 
    
    for i in range(num_train):
        # Genera un indice casuale tra 0 e l'ultimo indice disponibile rimasto
        random_index = random.randint(0, len(available_images) - 1)
        
        selected_img = available_images.pop(random_index)
        
        train_imgs.append(selected_img)

    # ESTRAZIONE VALIDATION SET
    print(f"--- Estrazione casuale di {num_val} immagini per il VALIDATION ---")
    for i in range(num_val):
        random_index = random.randint(0, len(available_images) - 1)
        selected_img = available_images.pop(random_index)
        val_imgs.append(selected_img)

    # ESTRAZIONE TEST SET 
    print(f"--- Estrazione casuale di {num_test} immagini per il TEST ---")
    for i in range(num_test):
        # Pesco dalle rimanenti
        random_index = random.randint(0, len(available_images) - 1)
        selected_img = available_images.pop(random_index)
        test_imgs.append(selected_img)
        
    print(f"\nRiepilogo Assegnazioni:")
    print(f"TRAIN: {len(train_imgs)} immagini")
    print(f"VAL:   {len(val_imgs)} immagini")
    print(f"TEST:  {len(test_imgs)} immagini")
    print(f"RIMASTE (non usate): {len(available_images)} immagini")

    # --- GENERAZIONE DEI FILE ---
    print("\n--- Generazione Tiles Training ---")
    tile_dataset(train_imgs, TRAIN_DIR, TRAIN_JSON, TILE_SIZE, STRIDE, coords_by_img)
    
    print("--- Generazione Tiles Validation ---")
    tile_dataset(val_imgs, VAL_DIR, VAL_JSON, TILE_SIZE, STRIDE, coords_by_img)

    print("--- Generazione Test Set (JSON) ---")
    with open(TEST_JSON, 'w') as f:
        json.dump(test_imgs, f)
    
    test_ground_truth = {img: coords_by_img[img] for img in test_imgs}
    with open(TEST_GT_JSON, 'w') as f:
        json.dump(test_ground_truth, f)


create_datasets(num_train=700, num_val=100, num_test=47)

# Modifica di Compose e ToTensor #
Modifico le seguenti classi per consentire il compose (e le transform) di img + annotations
prima avevo infatti che Compose accettava solo 1 param. : l'img, ora devo flippare anche le annotations

In [ ]:
# Classe Compose personalizzata che accetta (image, target)
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

# ToTensor che accetta (image, target)
class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

# Flip Orizzontale (Immagine + Box)
class RandomHorizontalFlip(object):
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            
            # Flippo l'immagine
            image = image.flip(-1)
            
            # Flippo i box
            bbox = target["boxes"]
            # x1_new = width - x2_old
            # x2_new = width - x1_old
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
        return image, target

# Flip Verticale (Immagine + Box) - Fondamentale per il satellite
class RandomVerticalFlip(object):
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            
            # Flippo l'immagine
            image = image.flip(-2)
            
            # Flippo i box
            bbox = target["boxes"]
            # y1_new = height - y2_old
            # y2_new = height - y1_old
            bbox[:, [1, 3]] = height - bbox[:, [3, 1]]
            target["boxes"] = bbox
        return image, target

# classe color Jitter per uniformare contrasto
class ColorJitter(object):
    def __init__(self, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05):
        # Usiamo l'implementazione veloce di PyTorch internamente
        self.transform = T.ColorJitter(brightness=brightness, 
                                       contrast=contrast, 
                                       saturation=saturation, 
                                       hue=hue)

    def __call__(self, image, target):
        # Applico il jitter SOLO all'immagine
        image = self.transform(image)
        
        # Restituisco l'immagine modificata e il target (box) INALTERATO
        return image, target



# Classe Dataset #

In [ ]:
class MioxViewDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        
        # Carico il file delle annotazioni
        with open(annotation_file, 'r') as f:
            self.imgs_data = json.load(f)

        # Mapping delle classi
        self.class_map = {11: 1, 12: 2, 13: 3, 15: 4, 17: 5, 18: 6, 19: 7, 20: 8,
            21: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14, 28: 15, 29: 16,
            32: 17, 33: 18, 34: 19, 35: 20, 36: 21, 37: 22, 38: 23, 40: 24,
            41: 25, 42: 26, 44: 27, 45: 28, 47: 29, 49: 30, 50: 31, 51: 32,
            52: 33, 53: 34, 54: 35, 55: 36, 56: 37, 57: 38, 59: 39, 60: 40,
            61: 41, 62: 42, 63: 43, 64: 44, 65: 45, 66: 46, 71: 47, 72: 48,
            73: 49, 74: 50, 76: 51, 77: 52, 79: 53, 83: 54, 84: 55, 86: 56,
            89: 57, 91: 58, 93: 59, 94: 60}

    def __len__(self):
        return len(self.imgs_data)

    def __getitem__(self, idx):
        data = self.imgs_data[idx]
        img_path = os.path.join(self.root_dir, data['img_name'])
        
        # Caricamento immagine
        img = Image.open(img_path).convert("RGB")
        
        boxes = []
        labels = []
        
        # --- COSTANTI PER FILTRAGGIO ---
        MIN_AREA = 16 * 16  
        MIN_DIM = 4         
        # -------------------------------

        for obj in data['objects']:
            xmin, ymin, xmax, ymax = obj['bounds']
            
            # Calcolo dimensioni
            width = xmax - xmin
            height = ymax - ymin
            area = width * height

            # Filtro Validità base
            if xmax <= xmin or ymax <= ymin:
                continue

            # Filtro Dimensioni Minime (< 4px)
            if width < MIN_DIM or height < MIN_DIM:
                continue

            # Filtro Area Minima (< 16x16) e Troncamento indiretto
            if area < MIN_AREA:
                continue

            # Check Label (Rimuovi classe 0 o non mappata)
            original_id = obj['type_id']
            label_id = self.class_map.get(original_id, 0) 
            
            if label_id == 0:
                continue

            # Se passa tutti i test, aggiungi
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label_id)

        # Conversione in Tensori
        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.tensor([idx])
        
        # Area e Iscrowd (Standard COCO)
        if len(boxes) > 0:
            target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * (target["boxes"][:, 2] - target["boxes"][:, 0])
        else:
            target["area"] = torch.zeros((0,), dtype=torch.float32)
            
        target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)

        # Gestione TILE VUOTE 
        if len(boxes) == 0:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["area"] = torch.zeros((0,), dtype=torch.float32)

        if self.transforms:
            img, target = self.transforms(img, target)
        else:
            img = F.to_tensor(img)

        return img, target

# Collate function
def collate_fn(batch):
    return tuple(zip(*batch))

# Modello #

Modifico la resnet per fare transfer learning

In [ ]:
def get_maskrcnn_model(num_classes):

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(
            weights="DEFAULT",
            min_size=512, # fisso la dimensione del resize automatico della rete (default 1333x800), altrimenti la rete effettua train su dimensioni diverse che dal test
            max_size=512, # analogo
            rpn_fg_iou_thresh=0.4, # considera roi se > 0.4
            rpn_bg_iou_thresh=0.2, # considera sfondo se < 0.2
        
            # aumento le proposte per essere sicuro di non perderne
            rpn_pre_nms_top_n_train=2000, 
            rpn_post_nms_top_n_train=2000,
            # ----------------------------------
        )    

    #le ancore della rete addestrata su COCO sono troppo grandi (32x32) rispetto a quelle che servono per xView -> inserisco anche 8x8 e 16x16 
        
    anchor_sizes = ((8,), (16,), (32,), (64,), (128,))
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    
    rpn_anchor_generator = AnchorGenerator(
        sizes=anchor_sizes, 
        aspect_ratios=aspect_ratios
    )
    
    model.rpn.anchor_generator = rpn_anchor_generator

    # SOSTITUZIONE  DELLA TESTA DELLA RPN 
    # Devo ricreare la "head" perché ora deve predire 15 ancore invece di 3.
    in_channels = model.rpn.head.cls_logits.in_channels
    
    # Calcoliamo quante ancore ci sono per ogni pixel (5 sizes * 3 ratios = 15)
    num_anchors = rpn_anchor_generator.num_anchors_per_location()[0]
    
    model.rpn.head = RPNHead(in_channels, num_anchors)
    
    # CAMBIO LA TESTA DEI BOX (Rettangoli + Classificazione)
    in_features_box = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features_box, num_classes)

    # CAMBIO LA TESTA DELLE MASCHERE (Segmentation), non ci serve a nulla -> None
    model.roi_heads.mask_predictor = None # fondamentale
    
    return model

# Transforms per data augmentation #

Altrimenti il modello funziona male (prove precedenti)

In [ ]:
def get_train_transform():
    return Compose([
        ToTensor(),                 # 1. Img -> Tensore
        
        # Data Augmentation Colore (Solo Train)
        # - Brightness/Contrast: 0.2-0.3 (gestisce ombre e luci forti)
        # - Hue: basso (0.05) perché un prato verde non diventa mai viola
        ColorJitter(),
        # 3. Data Augmentation Geometrica
        RandomHorizontalFlip(0.5),  
        RandomVerticalFlip(0.5)     
    ])

def get_val_transform():
    return Compose([
        ToTensor(), # In validazione niente Jitter né Flip
    ])

In [ ]:
# Dataset e Loader di TRAINING
train_dataset = MioxViewDataset(
    root_dir='/kaggle/working/train_tiles',
    annotation_file='/kaggle/working/train_annotations.json',
    transforms=get_train_transform()
)
train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True, 
    num_workers=2, collate_fn= collate_fn
)

# Dataset e Loader di VALIDATION
val_dataset = MioxViewDataset(
    root_dir='/kaggle/working/val_tiles',
    annotation_file='/kaggle/working/val_annotations.json',
    transforms=get_val_transform()
)
val_loader = DataLoader(
    val_dataset, batch_size=8, shuffle=False, 
    num_workers=2, collate_fn=collate_fn
)

#  Check delle tile dal train_loader #

In [ ]:
# Prendo un batch dal loader
images, targets = next(iter(train_loader))

# Prendo la prima immagine
img = images[0]
target = targets[0]

img = img.permute(1, 2, 0).numpy() # C,H,W -> H,W,C
img = np.clip(img, 0, 1)

plt.figure(figsize=(10, 10))
plt.imshow(img)
ax = plt.gca()

for box in target['boxes']:
    x1, y1, x2, y2 = box.numpy()
    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
    ax.add_patch(rect)

plt.title(f"Check Training Data: {len(target['boxes'])} boxes")
plt.show()

# Check classi nel val_dataset #

In [ ]:
TRAIN_ID_TO_NAME = {
    1: 'Fixed-wing Aircraft', 2: 'Small Aircraft', 3: 'Cargo Plane', 4: 'Helicopter',
    5: 'Passenger Vehicle', 6: 'Small Car', 7: 'Bus', 8: 'Pickup Truck',
    9: 'Utility Truck', 10: 'Truck', 11: 'Cargo Truck', 12: 'Truck w/Box',
    13: 'Truck Tractor', 14: 'Trailer', 15: 'Truck w/Flatbed', 16: 'Truck w/Liquid',
    17: 'Crane Truck', 18: 'Railway Vehicle', 19: 'Passenger Car', 20: 'Cargo Car',
    21: 'Flat Car', 22: 'Tank Car', 23: 'Locomotive', 24: 'Maritime Vessel',
    25: 'Motorboat', 26: 'Sailboat', 27: 'Tugboat', 28: 'Barge',
    29: 'Fishing Vessel', 30: 'Ferry', 31: 'Yacht', 32: 'Container Ship',
    33: 'Oil Tanker', 34: 'Engineering Vehicle', 35: 'Tower crane', 36: 'Container Crane',
    37: 'Reach Stacker', 38: 'Straddle Carrier', 39: 'Mobile Crane', 40: 'Dump Truck',
    41: 'Haul Truck', 42: 'Scraper/Tractor', 43: 'Front loader/Bulldozer', 44: 'Excavator',
    45: 'Cement Mixer', 46: 'Ground Grader', 47: 'Hut/Tent', 48: 'Shed',
    49: 'Building', 50: 'Aircraft Hangar', 51: 'Damaged Building', 52: 'Facility',
    53: 'Construction Site', 54: 'Vehicle Lot', 55: 'Helipad', 56: 'Storage Tank', 57: 'Shipping Container Lot',
    58: 'Shipping container', 59: 'Pylon', 60: 'Tower'
}

# Conto le classi nel validation set
val_class_counts = {}
for idx in range(len(val_dataset)):
    _, target = val_dataset[idx]
    labels = target['labels'].numpy()
    for l in labels:
        val_class_counts[l] = val_class_counts.get(l, 0) + 1

print("Classi presenti nel Validation Set:")
for class_id, count in sorted(val_class_counts.items()):
    name = TRAIN_ID_TO_NAME.get(class_id, str(class_id))
    print(f"{name}: {count}")

# Training rete #

In [ ]:
# CONFIGURAZIONE INIZIALE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Caricamento Modello
model = get_maskrcnn_model(num_classes=61) 
model.to(device)


# Congelamento Pesi
for param in model.parameters():
    param.requires_grad = True

# Blocco C1 
for param in model.backbone.body.conv1.parameters():
    param.requires_grad = False
for param in model.backbone.body.bn1.parameters():
    param.requires_grad = False

# Blocco C2 (Layer 1)
for param in model.backbone.body.layer1.parameters():
    param.requires_grad = False

# 3. Setup Optimizer e Scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
# 4. Setup Metrica MAP
metric = MeanAveragePrecision(iou_type="bbox", class_metrics=True).to(device)

# Storico
history = {
    'train_loss': [],
    'val_loss': [],
    'train_box_loss': [],
    'val_map_50': [], 
    'val_recall': []
}

num_epochs = 25


print("Inizio Training...")

for epoch in range(num_epochs):
    # ==========================
    # 1. FASE DI TRAINING
    # ==========================
    model.train()
    running_loss = 0.0
    running_box_loss = 0.0
    
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    
    for images, targets in train_loop:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass 
        loss_dict = model(images, targets)

        # Somma delle loss 
        losses = sum(loss for loss in loss_dict.values())
        
        box_loss = loss_dict['loss_box_reg']

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        # Accumulo
        running_loss += losses.item()
        running_box_loss += box_loss.item()
        
        # Aggiorno la barra di caricamento con la loss attuale
        train_loop.set_postfix(loss=losses.item())

    # Medie Training
    avg_train_loss = running_loss / len(train_loader)
    avg_box_loss = running_box_loss / len(train_loader)
    
    history['train_loss'].append(avg_train_loss)
    history['train_box_loss'].append(avg_box_loss)

    # ==========================
    # 2. FASE DI VALIDATION (Loss)
# Uso torch.no_grad() siccome non posso passare in modalità eval (non mi dà loss altrim), devo rimanere in train
    val_running_loss = 0.0
    

    with torch.no_grad():
        for images, targets in val_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_running_loss += losses.item()
            
    avg_val_loss = val_running_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)

    # ==========================
    # 3. FASE DI VALIDATION 
    # ==========================
    model.eval() # ORA passiamo in modalità valutazione per le predizioni
    
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f"Epoch {epoch+1} [Eval]"):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Predizioni
            predictions = model(images)
            
            # Aggiorno la metrica accumulando i dati
            metric.update(predictions, targets)


    val_metrics = metric.compute()
    
    map_50 = val_metrics['map_50'].item()
    recall = val_metrics['mar_100'].item()
    

    map_per_class = val_metrics['map_per_class'] 
    
    # Stampa Globale
    print(f"\n EPOCA {epoch+1}/{num_epochs}")
    print(f"   mAP @ 50 (Global): {map_50:.4f}")
    print(f"   Recall (Global):   {recall:.4f}")
    print("-" * 40)
    print(" DETTAGLIO CLASSI (Top & Flop):")

    # Creiamo una lista ordinata per vedere chi va bene e chi male
    class_results = []

    for i, score in enumerate(map_per_class):
        score_val = score.item()
        
        # Ignora classi con score -1 (che non c'erano nel validation set)
        if score_val == -1: continue
        
        # Recupera il nome
        class_name = TRAIN_ID_TO_NAME.get(i, f"Class {i}") 
        
        class_results.append((class_name, score_val))

    class_results.sort(key=lambda x: x[1], reverse=True) # Dall'alto al basso

    for name, score in class_results[:5]:
        print(f"{name:<20}: {score:.4f}")
    
    if len(class_results) > 10:
        print("      ...")
        for name, score in class_results[-5:]:
             print(f"{name:<20}: {score:.4f}")
    else:
        for name, score in class_results[5:]:
             print(f"      {name:<20}: {score:.4f}")

    # Salvataggio nello storico (Solo globali per non fare un JSON gigante)
    history['val_map_50'].append(map_50)
    history['val_recall'].append(recall)
    

    metric.reset()

    # ==========================
    # 4. REPORT E SALVATAGGIO
    # ==========================
    lr_scheduler.step(avg_val_loss)
    
    print(f"\n EPOCA {epoch+1}/{num_epochs}")
    print(f"   Train Loss: {avg_train_loss:.4f} | Box Loss: {avg_box_loss:.4f}")
    print(f"   Val Loss:   {avg_val_loss:.4f}")
    print(f"   mAP @ 50:   {map_50:.4f} (Obiettivo: > 0.5)")
    print(f"   Recall:     {recall:.4f}")
    print("-" * 40)
    
    # Salvo checkpoint
    torch.save(model.state_dict(), f'/kaggle/working/model_epoch_{epoch+1}.pth')
    
    # Salvo history JSON aggiornata
    with open('/kaggle/working/history.json', 'w') as f:
        json.dump(history, f)

    # Pulizia Memoria
    del images, targets, predictions, loss_dict, losses
    gc.collect()
    torch.cuda.empty_cache()

    current_lr = optimizer.param_groups[0]['lr']
    print(f"   Val Loss: {avg_val_loss:.4f} | Current LR: {current_lr:.2e}")

print("Training completato!")